In [1]:
import os
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


API Extracts

In [ ]:
df = []
p = 0

# Summer Path
paths = "/content/drive/MyDrive/Datasci_266_NLP/266_Project/266_full_dataset_parsed"

arr = os.scandir(paths)
print('Total number of files to scan:', len(os.listdir(paths)))

for file in arr:
    df2 = pd.read_csv(file.path, sep=",")
    p += 1
    df.append(df2)
df = pd.concat(df)
print('Processed', p, 'files.')
df = df.reset_index(drop=True)

Total number of files to scan: 277
Processed 277 files.


In [ ]:
df.head()

,corpusid,title,paperid,abstract,full_text
0,118966881,Homogeneous nucleation of ice,9a7a77296944d8ad22bc5a3aa914506de9e9d47c,5EFF8C5FBA4A4CB64AD4DE825AF3A8AarXiv:1901.1021...,"Homogeneous nucleation of ice January 30, 2019..."
1,90786940,Nitrate uptake of the red tide dinoflagellate ...,64455016a9e25a898ac1e820e09e0828ad25e72f,The ability of a red tide species to take up n...,Nitrate uptake of the red tide dinoflagellate ...
2,86848729,Novel Drug Delivery Systems for Rheumatoid Art...,7d65ad47283f0bef27063cca5acd9e178c37a3a3,Recent advances in science and technology radi...,Novel Drug Delivery Systems for Rheumatoid Art...
3,102380182,Uranium Particle Identification with SEM-EDX f...,91140522c35d80ea98e2b3f59b79eb222135c28d,Secondary ion mass spectrometry (SIMS) is a pr...,Uranium Particle Identification with SEM-EDX f...
4,23675512,Quantifying the degree of bias from using coun...,81874b0942dc1513cf29728aff5de5be3600f1f7,Citizen-science databases have been used to de...,Quantifying the degree of bias from using coun...


In [ ]:
# check for duplicates
df.duplicated().sum()

2915

In [ ]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [ ]:
# new length
len(df)

4186

In [ ]:
df.isna().sum()

corpusid     0
title        0
paperid      0
abstract     0
full_text    0
dtype: int64

In [ ]:
df = df[['abstract', 'full_text']]

In [ ]:
# removing newline characters that screw up the matching between abstract and full text
df['abstract'] = df['abstract'].str.replace("\n", " ")

# remove all text from start of string to end of abstract for each row
# find where abstract ends
# remove everything before that
df = df.drop([380]) #in spanish and one sentence that appears multiple times in full text
def remove_abstract(row):
    abstract = row['abstract']
    full_text = row['full_text']

    # Find the starting index of the abstract in the full_text
    start_index = full_text.find(abstract)

    clean_text = full_text[start_index + len(abstract):].strip()

    # some papers have the abstract added twice
    # checking again for abstract
    if abstract in clean_text:
      start_index2 = clean_text.find(abstract)
      clean_text2 = clean_text[start_index2 + len(abstract):].strip()
      return clean_text2
    else:
      return clean_text

# Apply the function to each row in the DataFrame
df['full_text_clean'] = df.apply(remove_abstract, axis=1)
df['full_text'] = df['full_text_clean'].copy()
df = df.drop(columns=['full_text_clean'])
df['abstract'][0] = df['abstract'][0][64:].strip() # additional cleaning
df.head()

,abstract,full_text
0,Ice nucleation is a process of great relevance...,ordinated water molecules. Ice nucleation fro...
1,The ability of a red tide species to take up n...,INTRODUCTION Red tides or harmful algal bloom...
2,Recent advances in science and technology radi...,RA is a systemic disorder characterized by inf...
3,Secondary ion mass spectrometry (SIMS) is a pr...,Introduction Secondary ion mass spectrometry ...
4,Citizen-science databases have been used to de...,aims in this study were to illustrate the effe...


In [ ]:
# we will have 1 row that should say "Yes", all others should be "No"
df1 = pd.DataFrame()
df1['Compare'] = df.apply(lambda x: 'Yes' if x['abstract'] in x['full_text'] else 'No',axis=1)
df1['Compare'].value_counts()

Compare
No     4184
Yes       1
Name: count, dtype: int64

In [ ]:
df.isna().sum()

abstract     0
full_text    0
dtype: int64

In [ ]:
#df_final =
df_final = df[['abstract', 'full_text']]
df_final.head()

,abstract,full_text
0,Ice nucleation is a process of great relevance...,ordinated water molecules. Ice nucleation fro...
1,The ability of a red tide species to take up n...,INTRODUCTION Red tides or harmful algal bloom...
2,Recent advances in science and technology radi...,RA is a systemic disorder characterized by inf...
3,Secondary ion mass spectrometry (SIMS) is a pr...,Introduction Secondary ion mass spectrometry ...
4,Citizen-science databases have been used to de...,aims in this study were to illustrate the effe...


In [ ]:
# df export / one csv

file_name = '/content/drive/MyDrive/Datasci_266_NLP/266_Project/envisci_articles.csv'
df_final.to_csv(file_name, index=False, header=True)
print('Done.')

Done.


In [ ]:
# pulling old copy and removing nulls
df_nulls = pd.read_csv('/content/drive/MyDrive/Datasci_266_NLP/266_Project/envisci_articles.csv')

In [ ]:
df_polished = df_nulls.dropna()

file_name = '/content/drive/MyDrive/Datasci_266_NLP/266_Project/envisci_articles.csv'
df_polished.to_csv(file_name, index=False, header=True)
print('Done.')

Done.
